In [1]:
!pip install requests

In [ ]:
import requests
from getpass import getpass

# APIキー入力
api_key = getpass('APIキーを入力してください: ')

# 駅名入力
search_station = input('検索する駅名を入力してください: ')

# --- 駅名から緯度経度を取得 ---
geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'

geocode_response = requests.get(
    geocode_url,
    params={
        'address': search_station,
        'language': 'ja',
        'key': api_key
    }
)

geocode_data = geocode_response.json()

if geocode_data['status'] != 'OK':
    print(f"地名の取得に失敗しました: {geocode_data['status']}")
    if 'error_message' in geocode_data:
        print(f"メッセージ: {geocode_data['error_message']}")
    exit()

# 緯度経度を取得
location = geocode_data['results'][0]['geometry']['location']
latitude = location['lat']
longitude = location['lng']

print(f"\n📍 {search_station} の位置: 緯度={latitude}, 経度={longitude}\n")

# --- 近隣のレストラン検索 ---
places_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'location': f"{latitude},{longitude}",
    'radius': 500,  # 検索半径（メートル）
    'type': 'restaurant',
    'language': 'ja',
    'key': api_key
}

places_response = requests.get(places_url, params=params)
places_data = places_response.json()

if places_data['status'] == 'OK':
    print(f"🍽 近隣のレストラン情報 ({params['radius']}m以内):\n")
    if places_data['results']:
        for place in places_data['results']:
            name = place.get('name', 'N/A')
            rating = place.get('rating', '評価なし')
            address = place.get('vicinity', '住所なし')

            print(f"  レストラン名: {name}")
            print(f"  評価: {rating}")
            print(f"  住所: {address}")
            print("-" * 30)
    else:
        print("指定された半径内にレストランは見つかりませんでした。")
elif places_data['status'] == 'ZERO_RESULTS':
    print("指定された半径内にレストランは見つかりませんでした。")
else:
    print(f"エラー: {places_data['status']}")
    if 'error_message' in places_data:
        print(f"メッセージ: {places_data['error_message']}")
